In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [9]:
np.random.seed(0)
data = pd.read_csv("Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("Data\\cle\\Rand_valid.csv", sep=',')

In [10]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [11]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [12]:
from fit9_nn import fit, comb_error

In [46]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias=True)
        self.fc12 = nn.Linear(24, 1, bias=True)
        
        self.fc21 = nn.Linear(16, 24)
        self.fc22 = nn.Linear(24, 5)
        self.fc23 = nn.Linear(5, 1)
        
        self.fc31 = nn.Linear(2,2, bias=False)
        
    def disc_activation(self, h):
        v1 = torch.sigmoid(h[:,0])
        v2 = torch.exp(h[:,1])
        return torch.stack((v1, v2), dim=1)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)
        
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2))
        x2 = self.fc23(x2)              
        
        x3 = torch.cat((x1, x2), 1)
        x3 = self.fc31(x3)
        x3 = self.disc_activation(x3)
        return x3

In [48]:
n_epochs = 200
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = 'Data\\checkpoints\\mytraining4.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [49]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [50]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [51]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [52]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining4.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, 
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 1.0698892672856648, Validation Loss: 289.31280517578125
Epoch 2: Training Loss: 1.061783492565155, Validation Loss: 289.2580871582031
Epoch 3: Training Loss: 1.053925096988678, Validation Loss: 289.1998291015625
Epoch 4: Training Loss: 1.0462522010008495, Validation Loss: 289.1378479003906
Epoch 5: Training Loss: 1.0388684372107189, Validation Loss: 289.07183837890625
Epoch 6: Training Loss: 1.031759689251582, Validation Loss: 289.0021057128906
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 1.0249740183353424, Validation Loss: 288.9285583496094
Epoch 8: Training Loss: 1.0182409882545471, Validation Loss: 288.8509216308594
Epoch 9: Training Loss: 1.011907547712326, Validation Loss: 288.76837158203125
Epoch 10: Training Loss: 1.0058911343415577, Validation Loss: 288.6802978515625
Epoch 11: Training Loss: 0.9999982019265493, Validation Loss: 288.5868835449219
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9944183528423309, Validation Loss: 288.48

Epoch 97: Training Loss: 0.8669556776682535, Validation Loss: 278.07354736328125
Epoch 98: Training Loss: 0.866313636302948, Validation Loss: 277.9292297363281
Epoch 99: Training Loss: 0.8656964600086212, Validation Loss: 277.76055908203125
Epoch 100: Training Loss: 0.8650820056597391, Validation Loss: 277.5957336425781
Epoch 101: Training Loss: 0.8644820352395376, Validation Loss: 277.43096923828125
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.863837351401647, Validation Loss: 277.2635192871094
Epoch 103: Training Loss: 0.8632444043954214, Validation Loss: 277.093017578125
Epoch 104: Training Loss: 0.862636516491572, Validation Loss: 276.89276123046875
Epoch 105: Training Loss: 0.8620173533757528, Validation Loss: 276.6861267089844
Epoch 106: Training Loss: 0.8614141543706259, Validation Loss: 276.47808837890625
New Checkpoint Saved into PATH
Epoch 107: Training Loss: 0.8608196477095286, Validation Loss: 276.2853698730469
Epoch 108: Training Loss: 0.8602584898471832, Val

Epoch 195: Training Loss: 0.8069517115751902, Validation Loss: 278.5416564941406
Epoch 196: Training Loss: 0.8066899379094442, Validation Loss: 278.4396057128906
Epoch 197: Training Loss: 0.8064996699492136, Validation Loss: 278.38232421875
Epoch 198: Training Loss: 0.806254376967748, Validation Loss: 277.98394775390625
Epoch 199: Training Loss: 0.8060455322265625, Validation Loss: 277.4555358886719
Epoch 200: Training Loss: 0.8058696190516154, Validation Loss: 277.2539367675781
